# Synthesizing Benchmarks for Predictive Modeling

[Chris Cummins](http://chriscummins.cc/),
[Pavlos Petoumenos](http://homepages.inf.ed.ac.uk/ppetoume/),
[Zheng Wang](http://www.lancaster.ac.uk/staff/wangz3/),
[Hugh Leather](http://homepages.inf.ed.ac.uk/hleather/).

**Abstract**
> Predictive modeling using machine learning is an effective method for building
> compiler heuristics, but there is a shortage of benchmarks. Typical machine
> learning experiments outside of the compilation field train over thousands or
> millions of examples. In machine learning for compilers, however, there are
> typically only a few dozen common benchmarks available. This limits the
> quality of learned models, as they have very sparse training data for what are
> often high-dimensional feature spaces. What is needed is a way to generate an
> unbounded number of training programs that finely cover the feature space. At
> the same time the generated programs must be similar to the types of programs
> that human developers actually write, otherwise the learning will target the
> wrong parts of the feature space.
>
> We mine open source repositories for program fragments and apply deep learning
> techniques to automatically construct models for how humans write programs. We
> then sample the models to generate an unbounded number of runnable training
> programs, covering the feature space ever more finely. The quality of the
> programs is such that even human developers struggle to distinguish our
> generated programs from hand-written code.
>
> We use our generator for OpenCL programs, CLgen, to automatically synthesize
> thousands of programs and show that learning over these improves the
> performance of a state of the art predictive model by $1.27\times$. In
> addition, the fine covering of the feature space automatically exposes
> weaknesses in the feature design which are invisible with the sparse training
> examples from existing benchmark suites. Correcting these weaknesses further
> increases performance by $4.30\times$.

**Keywords**  Synthetic program generation, OpenCL, Benchmarking, Deep Learning, GPUs

This is a live research artifact. See [here](https://github.com/ChrisCummins/paper-synthesizing-benchmarks) for instructions to install this artifact on your own hardware. Please note that performance numbers obtained on hardware different to what we used may differ from those in the paper.

In [ ]:
# preamble
%load_ext autoreload
%autoreload 2
%matplotlib inline
%run lib/preamble.py

from IPython.core.display import HTML
HTML("""<style>
/* hide index column in Pandas Data Frames */
table.dataframe thead th:first-child { display: none; }
table.dataframe tbody th { display: none; }

/* tweak text size */
div.text_cell_render { font-size: 1.3em; line-height: 1.4em; }
</style>""")

# 1. Introduction

Predictive modeling is a well researched method for building optimization
heuristics that often exceed human experts and reduces development time. A set
of training programs are identified which are expected to be representative of
the application domain. The programs are compiled and executed with different
parameter values for the target heuristic, to determine which are the best
values for each training program. Each program is also summarized by a vector of
features which describe the information that is expected to be important in
predicting the best heuristic parameter values. These training examples of
program features and desired heuristic values are used to create a machine
learning model which, when given the features from a new, unseen program, can
predict good heuristic values for it.

It is common for the feature vectors to contain dozens of elements. This means
that a large volume of training data is needed to have an adequate sampling over
the feature space. Without it, the machine learned models can only capture the
coarse characteristics of the heuristic, and new programs which do not lie near
to training points may be wrongly predicted. The accuracy of the machine learned
heuristic is thus limited by the sparsity of the training points.

There have been efforts to solve this problem using templates. The essence of
the approach is to construct a probabilistic grammar with embedded semantic
actions that defines a language of possible programs. New programs may be
created by sampling the grammar and, through setting probabilities on the
grammar productions, the sampling is biased towards producing programs from one
part of the space or another. This technique is potentially completely general,
since a grammar can theoretically be constructed to match any desired program
domain. However, despite being theoretically possible, it is not easy to
construct grammars which are both suitably general and also produce programs
that are in any way similar to human written programs. It has been shown to be
successful over a highly restricted space of stencil benchmarks with little
control flow or program variability. But, it is not clear how much effort it
will take, or even if it is possible for human experts to define grammars
capable of producing human like programs in more complex domains.

By contrast, our approach does not require an expert to define what human
programs look like. Instead, we automatically infer the structure and likelihood
of programs over a huge corpus of open source projects. From this corpus, we
learn a probability distribution over sets of characters seen in human written
code. Later, we sample from this distribution to generate new random programs
which, because the distribution models human written code, are indistinguishable
from human code. We can then populate our training data with an unbounded number
of human like programs, covering the space with any desired granularity, far
more finely than either existing benchmark suites, or even the corpus of open
source projects. Our approach is enabled by two recent developments:

The first is the breakthrough effectiveness of deep learning for modeling
complex structure in natural languages. As we show, deep learning is capable not
just of learning the macro syntactical and semantic structure of programs, but
also the nuances of how humans typically write code. It is truly remarkable when
one considers that it is given no prior knowledge of the syntax or semantics of
the language.

The second is the increasing popularity of public and open platforms for hosting
software projects and source code. This popularity furnishes us with the
thousands of programming examples that are necessary to feed into the deep
learning. These open source examples are not, sadly, as useful for directly
learning the compiler heuristics since they are not presented in a uniform,
runnable manner, nor do they typically have extractable test data. Preparing
each of the thousands of open source projects to be directly applicable for
learning compiler heuristics would be an insurmountable task. In addition to our
program generator, CLgen, we also provide an accompanying host driver which
generates datasets for, then executes and profiles synthesized programs.

We make the following contributions:

-   We are the first to apply deep learning over source codes to
    synthesize compilable, executable benchmarks.

-   A novel tool CLgen for general-purpose benchmark synthesis using
    deep learning. CLgen automatically generates thousands of human like
    programs for use in predictive modeling.

-   We use CLgen to automatically improve the performance of a state of
    the art predictive model by 1.27x, and expose limitations in
    the feature design of the model which, after correcting, further
    increases performance by 2.66x.

# 2. Motivation

In this section we make the argument for synthetic benchmarks. We identified
frequently used benchmark suites in a survey of research papers in the field
of GPGPU performance tuning from top tier conferences between 2013–2016: CGO,
HiPC, PACT, and PPoPP:

In [ ]:
data = pd.read_csv("../data/lit-review.csv")
data

Analysis of results:

In [ ]:
benchmark_counts = [x[1] for x in data.groupby("Paper").size().iteritems()]
suites = pd.DataFrame(
    [(x[0], x[1] / labmath.mean(benchmark_counts))
     for x in data.groupby(["Benchmark Suite"]).size().iteritems()],
    columns=["Benchmark Suite", "#. benchmarks per paper"])
suites.sort_values("#. benchmarks per paper", inplace=True, ascending=False)

print("Average number of benchmarks per paper:", round(labmath.mean(benchmark_counts), 2))
print("Median number of benchmarks per paper: ", round(labmath.median(benchmark_counts), 2))
print()
suites

**Figure 2:**  The average number of benchmarks used in GPGPU research papers, organized by origin. In this work we use the seven most popular benchmark suites.

In [ ]:
ax = sns.barplot(x="Benchmark Suite", y="#. benchmarks per paper", data=suites)
# threshold for benchmark suites used in paper:
plt.axhline(y=.5, color="k", lw=1)
plt.setp(ax.get_xticklabels(), rotation=90)
plt.xlabel(""); plt.ylabel("#. benchmarks used")
viz.finalise(figsize=(4, 2.25), tight=True)
complete(msg="Replicated analysis of lit. review")

We selected the 7 most frequently used benchmark suites (used in 92% of
results), and evaluated the performance of the state of the art *Grewe et al.*
predictive model across each. Benchmark data:

In [ ]:
amd_benchmarks = cgo13.LabelledData.from_csv('../data/amd-benchmarks.csv', group_by="suite")
amd_benchmarks

In [ ]:
nvidia_benchmarks = cgo13.LabelledData.from_csv('../data/amd-benchmarks.csv', group_by="suite")
nvidia_benchmarks

The *Grewe et al.* model predicts whether running a given OpenCL kernel on the GPU gives better
performance than on the CPU. The performance of a model trained on one benchmark
suite and used to predict the optimal mapping for another suites is generally
very poor.

**Table 1:** Performance relative to the optimal of the *Grewe et al.* predictive model across different benchmark suites on an AMD GPU. The columns show the suite used for training; the rows show the suite used for testing.

In [ ]:
data = amd_benchmarks
metrics = cgo13.classification(data, group_by="suite")
results = np.zeros((len(metrics), len(metrics[0])))

for j in range(len(metrics)):
    for i in range(len(metrics[j])):
        results[j, i] = metrics[j][i].oracle * 100  # % optimal
results = results.T

groups = sorted(set(data["Group"]))

ax = sns.heatmap(results, annot=True, fmt=".0f", linewidths=.5,
                 xticklabels=shortlabels(groups),
                 yticklabels=shortlabels(groups), cbar=False,
                 cmap="RdBu", center=50, vmin=0, vmax=100)
for t in ax.texts:
    t.set_text(t.get_text() + "%")
plt.xlabel("Benchmark suite used for training")
plt.ylabel("Benchmark suite used for testing")
viz.finalise(figsize=(6, 6), tight=True)
complete(msg="Replicated cross-validation of predictive model across benchmark suites")

The benchmark suite which provides the best results, NVIDIA SDK,
achieves on average only 49% of the optimal performance. The worst case is when
training with Parboil to predict the optimal mappings for Polybench, where the
model achieves only 11.5% of the optimal performance. It is clear that
heuristics learned on one benchmark suite fail to generalize across other
suites. This problem is caused both by the limited number of benchmarks contained in
each suite, and the distribution of benchmarks within the feature space.

**Figure 3a**: A two dimensional projection of the Grewe
et al. predictive model over Parboil benchmarks on
an NVIDIA GPU.

In [ ]:
clf = cgo13.model()
parboil = nvidia_benchmarks[nvidia_benchmarks["benchmark"].str.contains(r"^parboil-")]

B_out = cgo13.xval_benchmarks(clf, parboil)
num_correct = sum(B_out["p_correct"])

pca = PCA(n_components=2)
pca.fit(cgo13.raw_features(parboil))

X = pca.transform(cgo13.raw_features(parboil))

ax = plot_pca(X, B_out)

xlim = ax.get_xlim()  # we'll use these for the next plot
ylim = ax.get_ylim()

viz.finalise(figsize=(2.5, 2.5), tight=True)

As we see, there is a dense cluster of neighboring benchmarks, a smaller cluster
of three benchmarks, and two outliers. The lack of neighboring observations
means that the model is unable to learn a good heuristic for the two outliers,
which leads to them being incorrectly optimized. Now we train the predictive
model with additional benchmarks which are neighboring in the feature space.

**Figure 3b:** Two outliers in (a) are incorrectly predicted due to the lack of nearby observations. The additional neighboring observations in (b) corrects this.

In [ ]:
clf = clone(clf)  # reset model

additional_benchmarks = [
    "shoc-1.1.5-FFT-ifft1D_512",
    "shoc-1.1.5-S3D-ratt6_kernel",
    "shoc-1.1.5-S3D-ratx2_kernel",
    "shoc-1.1.5-S3D-ratt9_kernel",
]

additional = nvidia_benchmarks[nvidia_benchmarks["benchmark"].str.contains(
    "|".join(additional_benchmarks))]
parboil_and_additional = nvidia_benchmarks[nvidia_benchmarks["benchmark"].str.contains(
    r"^{}|{}".format("parboil", "|".join(additional_benchmarks)))]

B_out = cgo13.xval_benchmarks(clf, parboil_and_additional, prefix="parboil")

num_correct_with_additional = sum(B_out["p_correct"])

X = pca.transform(cgo13.raw_features(parboil))

ax = plot_pca(X, B_out, additional, pca)

ax.set_xlim(xlim)  # use same axis as before
ax.set_ylim(ylim)

viz.finalise(figsize=(2.5, 2.5), tight=True)
complete(num_correct_with_additional > num_correct,
         "Neighboring observations improves classification accuracy")

The addition of these observations (and the information they provide about that
part of the feature space) causes the two outliers to be correctly optimized. We
found such outliers in all of the benchmark suites.

These results highlight the significant affect that the number and distribution
of training programs has on the quality of predictive models. Without good
coverage of the feature space, any machine learning methodology is unlikely to
produce high quality heuristics, suitable for general use on arbitrary real
applications. Our novel approach, described in the next section, solves this
problem by generating an unbounded number of programs to cover the feature space
at any desired granularity.

# 3. Overview of Our Approach

In this paper we present CLgen, a tool for synthesizing OpenCL benchmarks, and
an accompanying host driver for executing synthetic benchmarks for gathering
performance data for predictive modeling. While we demonstrate our approach
using OpenCL, it is language agnostic. Our tool CLgen learns the semantics and
structure from over a million lines of hand-written code from GitHub, and
synthesizes programs through a process of iterative model sampling. We then use
a host driver to execute the synthesized programs to gather performance data for
use in predictive modeling. Our approach extends the state of the art by
providing a general-purpose solution for benchmark synthesis, leading to better
and more accurate predictive models.

In the course of evaluating our technique against prior work we discovered that
it is also useful for evaluating the quality of features. Since we are able to
cover the space so much more finely than the prior work, which only used
standard benchmark suites, we are able to find multiple programs with identical
feature values but different best heuristic values. This indicates that the
features are not sufficiently discriminative and should be extended with more
information to allow those programs to be separated. We go on to show that doing
this significantly increases the performance of the learned heuristics. We
expect that our technique will be valuable for feature designers.

# 4. CLgen: Benchmark Synthesis

CLgen is an undirected, general-purpose program synthesizer for OpenCL. It
adopts and augments recent advanced techniques from deep learning to learn over
massive codebases. In contrast to existing grammar and template based
approaches, CLgen is entirely probabilistic. It *learns* to program using neural
networks which model the semantics and usage of a huge corpus of code fragments
in the target programming language. This section describes the assembly of an
OpenCL language corpus, the application of deep learning over this corpus, and
the process of synthesizing programs.

## 4.1. An OpenCL Language Corpus

Deep learning requires large datasets. For the purpose of modeling a programming
language, this means assembling a very large collection of real, hand-written
source codes. We assembled OpenCL codes by mining public repositories on the
popular code hosting site GitHub.

This is itself a challenging task since OpenCL is an embedded language, meaning
device code is often difficult to untangle and GitHub does not presently
recognize it as a searchable programming language. We developed a search engine
for the GitHub API which attempts to identify and download standalone OpenCL
files through a process of file scraping and recursive header inlining.

The dataset we mined (from June 2016):

In [ ]:
import clgen.dbutil

data = "../data/github.db"
print("# content files:", clgen.dbutil.num_rows_in(data, "ContentFiles"))
print("line count:     ", clgen.dbutil.lc(data, "ContentFiles"))

Randomly selected file *(replay cell to select a new file)*:

In [ ]:
db = clgen.dbutil.connect("../data/github.db").cursor()
db.execute("SELECT Contents FROM ContentFiles ORDER BY RANDOM() LIMIT 1")
print(db.fetchone()[0])
db.close()

We prune the raw dataset extracted from GitHub using a custom toolchain we
developed for rejection filtering and code rewriting, built on LLVM.

In [ ]:
# To replicate the mining of GitHub for OpenCL files:
#
# Generate an API token by following the instructions here:
#   https://help.github.com/articles/creating-an-access-token-for-command-line-use/>
# 
# From the command line, run:
#
#   $ export GITHUB_USERNAME=[your-username]
#   $ export GITHUB_PW=[your-password]
#   $ export GITHUB_TOKEN=[your-api-token]
#   $ clgen-create-db --github github.db
#   $ clgen-fetch-github github.db
#
# This process issues thousands of GitHub API requests per minute over the course of several hours.
# Please exercise restrained in minimizing your use of this program -- we don't want to upset
# the nice folks at GH :-)

#### Rejection Filter

The rejection filter accepts as input a content file and returns whether or not
it contains compilable, executable OpenCL code. To do this we attempt to compile
the input to NVIDIA PTX bytecode and perform static analysis to ensure a minimum
static instruction count of three. We discard any inputs which do not compile or
contain fewer than three instructions.

Bytecode features of an example program:

In [ ]:
bc = clgen.preprocess.compile_cl_bytecode("""
#define DTYPE float
#define ALPHA(a) 3.5f * a
inline DTYPE ax(DTYPE x) { return ALPHA(x); }

__kernel void saxpy( /* SAXPY kernel */
    __global DTYPE *input1,
    __global DTYPE *input2,
    const int nelem)
{
    unsigned int idx = get_global_id(0);
    // = ax + y
    if (idx < nelem) {
        input2[idx] += ax(input1[idx]); }}
""")
        
# extract features
features = clgen.preprocess.bytecode_features(bc)

print(clgen.format_json(features))
complete("instructions_of_all_types" in features, "Bytecode feature extraction")

During initial development it became apparent that isolating the OpenCL device
code leads to a higher-than-expected discard rate (that is, seemingly valid
OpenCL files being rejected). Through analyzing 148k lines of compilation
errors, we discovered a large number of failures caused by undeclared
identifiers — a result of isolating device code — 50% of undeclared identifier
errors in the GitHub dataset were caused by only 60 unique identifiers. To
address this, we developed a *shim header* which contains inferred values for
common type definitions (e.g. `FLOAT_T`), and common constants (e.g. `WGSIZE`).

**Listing 1:** The shim header file, providing inferred type aliases and constants for OpenCL on GitHub.

In [ ]:
import clgen.native
print(readfile(clgen.native.SHIMFILE))

In [ ]:
# without shim:
try:
    clgen.preprocess.preprocess("""
        __kernel void foo(__global float* a) {
            a[get_global_id(0)] = ZERO;
        }
    """, use_shim=False)
    complete(False, "Code fails to compile without shim header")
except clgen.preprocess.BadCodeException:
    complete(True, "Code fails to compile without shim header")

In [ ]:
code = clgen.preprocess.preprocess("""
    __kernel void foo(__global float* a) {
        a[get_global_id(0)] = ZERO;
    }
""", use_shim=True)

print(code)
complete(msg="Code compiles with shim header")

Injecting the shim decreases the discard rate from 40% to 32%, responsible for
an additional 88k lines of code in the final language corpus. The resulting
dataset is 2.0 million lines of compilable OpenCL source code.

In [ ]:
def linecount_of_good_files(data):
    db = clgen.dbutil.connect(data).cursor()
    db.execute("""SELECT LC(ContentFiles.Contents) FROM ContentFiles
               LEFT JOIN PreprocessedFiles ON ContentFiles.id=PreprocessedFiles.id
               WHERE PreprocessedFiles.status=0""")
    return db.fetchone()[0]

data = "../data/github.db"

# preprocess dataset
# note results are cached. empty cache by uncommenting the next line (will take >30min):
# clgen.dbutil.remove_preprocessed(data)
clgen.preprocess.preprocess_db(data)

num_good_files = clgen.dbutil.num_rows_in(data, "PreprocessedFiles", condition="WHERE status=0")
num_bad_files = clgen.dbutil.num_rows_in(data, "PreprocessedFiles", condition="WHERE status!=0")
ratio_bad = num_bad_files / (num_good_files + num_bad_files)

print("# files in final dataset:", num_good_files)
print("# rejected files:        ", num_bad_files, "({:.0f}%)".format(ratio_bad * 100))
print("line count of good files:", linecount_of_good_files(data))

#### Code Rewriting

Programming languages have few of the issues of semantic interpretation present
in natural language, though there remains many sources of variance at the
syntactic level. For example, the presence and content of comments in code, and
the choice of identifying names given to variables. We consider these
ambiguities to be *non-functional variance*, and developed a tool to normalize
code of these variances so as to make the code more amenable to machine
learning. This is a three step process:

1.  The source is pre-processed to remove macros, conditional compilation, and
    source comments.

2.  Identifiers are rewritten to have a short but unique name based on their
    order of appearance, using the sequential series $\{a, b, c,
    \ldots, aa, ab, ac, \ldots\}$
    for variables and $\{A, B, C,
    \ldots, AA, AB, AC, \ldots\}$
    for functions. This process isolates the syntactic structure of the code,
    and unlike prior work, our rewrite method preserves program behavior.
    Language built-ins (e.g. `get_global_id`, `asin`) are not rewritten.

3.  A variant of the Google C++ code style is enforced to ensure consistent use
    of braces, parentheses, and white space.
    
**Figure 5:**  The code rewriting process, which transforms code to make it more amenable to language modeling.

In [ ]:
try:
    print(clgen.preprocess.preprocess("""
#define DTYPE float
#define ALPHA(a) 3.5f * a
inline DTYPE ax(DTYPE x) { return ALPHA(x); }

__kernel void saxpy( /* SAXPY kernel */
    __global DTYPE *input1,
    __global DTYPE *input2,
    const int nelem)
{
    unsigned int idx = get_global_id(0);
    // = ax + y
    if (idx < nelem) {
        input2[idx] += ax(input1[idx]); }}
"""))
    complete(True, "Preprocessing OpenCL code for machine learning")
except Exception as e:
    complete(False, "Preprocessing OpenCL code for machine learning")

Randomly selected preprocessed file from dataset *(replay cell to select a new file)*:

In [ ]:
db = clgen.dbutil.connect("../data/github.db").cursor()
db.execute("SELECT Contents FROM PreprocessedFiles WHERE status=0 ORDER BY RANDOM() LIMIT 1")
print(db.fetchone()[0])
db.close()

A side effect of this process is a reduction in code size, largely due to the
removal of comments and excess white space. The final language corpus contains
1.3 million lines of transformed OpenCL, consisting of 9487 kernel functions.
Identifier rewriting reduces the bag-of-words vocabulary size by 84%.

In [ ]:
import clgen.train

db = clgen.dbutil.connect("../data/github.db")
clgen.train.create_corpus(db, "../data/corpus.cl", gh=True)

lc, wc, cc = line_word_char_count("../data/corpus.cl")
print("  #. lines:", lc)
print("  #. words:", wc)
print("  #. chars:", cc)
complete(msg="Analysis of GitHub dataset")

## 4.2. Learning OpenCL

Generating valid, executable program code is an ambitious and
challenging goal for machine learning. We employ state of the art deep
language modeling techniques to achieve this task.

We use the Long Short-Term Memory (LSTM) architecture of Recurrent Neural
Network to learn a character-level language model over the corpus of OpenCL
compute kernels. The LSTM network architecture comprises recurrent layers of
*memory cells*, each consisting of an input, output, and forget gate, and an
output layer providing normalized probability values from a 1-of-K coded
vocabulary.

We use a 3-layer LSTM network with 2048 nodes per layer. We train this
17-million parameter model using *Stochastic Gradient Descent* for 50 epochs,
using an initial learning rate of 0.002, decaying by a factor of one half every
5 epochs. Training took three weeks on a single machine using an NVIDIA GTX
Titan, with a final model size of 648MB. Training the network is a one-off cost,
and can be parallelized across devices. The trained network can be deployed to
lower-compute machines for use.

In [ ]:
import clgen.model

print("Training a tiny network on a tiny dataset:")
tiny_model = clgen.model.from_json({
        "corpus": {
            "path": "../data/tiny-corpus"
        },
        "train_opts": {
            "model_type": "lstm",
            "rnn_size": 128,
            "num_layers": 2,
            "max_epochs": 10
        }
    })
tiny_model.cache.empty()
print(tiny_model.corpus)
tiny_model.train()
tiny_model

In [ ]:
print("The model used in the paper (pre-trained):")
model = clgen.model.from_tar("../data/clgen-github-model-2016-nov-2048x3.tar.bz2")
model

## 4.3. Synthesizing OpenCL

We synthesize OpenCL compute kernels by iteratively sampling the learned
language model. We implemented two modes for model sampling: the first involves
providing an *argument specification*, stating the data types and modifiers of
all kernel arguments. When an argument specification is provided, the model
synthesizes kernels matching this signature. In the second sampling mode this
argument specification is omitted, allowing the model to synthesize compute
kernels of arbitrary signatures, dictated by the distribution of argument types
within the language corpus.

In either mode we generate a *seed* text:

In [ ]:
print(clgen.sampler.serialize_argspec([
    '__global float*',
    '__global float*',
    '__global float*',
    'const int'
]))

In [ ]:
print(clgen.sampler.serialize_argspec([
    '__global int*',
    'const int'
]))

In [ ]:
print(clgen.sampler.serialize_argspec([
    '__global int*',
    'const int'
]))

We then sample the model, character by character, until the end of the compute kernel is reached, or until a
predetermined maximum number of characters is reached.

Live demos (re-run to re-sample):

In [ ]:
import clgen.sampler

argspec = ['__global float*', '__global float*', '__global float*', 'const int']

sampler = clgen.sampler.from_json({
        "kernels": { 
            "args": argspec,
            "max_length": 300,
        },
        "sampler": {
            "batch_size": 1,
            "max_batches": 1
        }
    })

print("Sample from the tiny model we just trained:")
sampler.sample(tiny_model)

In [ ]:
print("Sample from the model used in the paper:")
sampler.sample(model)

The rejection filter then either accepts or rejects the sample as a candidate synthetic benchmark

Live demo which generates a single statically-verified OpenCL kernel using the pre-trained network:

In [ ]:
print("Generate a statically checked kernel using the pre-trained network:")
sampler = clgen.sampler.from_json({
        "kernels": { 
            "args": argspec,
            "max_length": 500,
        },
        "sampler": {
            "batch_size": 5,
            "max_kernels": 1
        }
    })

sampler.cache(model).empty()
sampler.sample(model)
print("done.")

# 5. Benchmark Execution

We developed a host driver to gather performance data from synthesized CLgen
code. The driver accepts as input an OpenCL kernel, generates *payloads* of
user-configurable sizes, and executes the kernel using the generated payloads,
providing dynamic checking of kernel behavior.

In [ ]:
import clgen.cldrive, clgen.preprocess

src = clgen.preprocess.preprocess("""
__kernel void benchmark(__global float* a, __global float* b, const float c, const int n) {
   const int i = get_global_id(0);
   if (i < n)
     a[i] += c * b[i];
}
""")

if has_opencl():
    ctx, queue = clgen.cldrive.init_opencl()
    driver = clgen.cldrive.KernelDriver(ctx, src)
    print(driver)
else:
    print("no OpenCL support, skipped", file=sys.stderr)

## 5.1. Generating Payloads

A *payload* encapsulates all of the arguments of an OpenCL compute kernel. After
parsing the input kernel to derive argument types, a rule-based approach is used
to generate synthetic payloads. For a given global size $S_g$: host buffers of
$S_g$ elements are allocated and populated with random values for global pointer
arguments, device-only buffers of $S_g$ elements are allocated for local pointer
arguments, integral arguments are given the value $S_g$, and all other scalar
arguments are given random values. Host to device data transfers are enqueued
for all non-write-only global buffers, and all non-read-only global buffers are
transferred back to the host after kernel execution.

In [ ]:
if has_opencl():
    size = 4096
    payload = clgen.cldrive.KernelPayload.create_random(driver, size)
    print(payload)
else:
    print("no OpenCL support, skipped", file=sys.stderr)

## 5.2. Dynamic Checker

For the purpose of performance benchmarking we are not interested in the
correctness of computed values, but we define a class of programs as performing
*useful work* if they predictably compute some result. We devised a low-overhead
runtime behavior check to validate that a synthesized program does useful work
based on the outcome of four executions of a tested program:

**Step 1:** Create 4 equal size payloads $A_{1in}$, $B_{1in}$, $A_{2in}$, $B_{2in}$,
    subject to restrictions: $A_{1in}=A_{2in}$, $B_{1in}=B_{2in}$, $A_{1in} \ne
    B_{1in}$.

In [ ]:
from copy import deepcopy

if has_opencl():
    # create payloads
    A1in = clgen.cldrive.KernelPayload.create_sequential(driver, size)
    A2in = deepcopy(A1in)

    B1in = clgen.cldrive.KernelPayload.create_random(driver, size)
    B2in = deepcopy(B1in)

    # check contstraints
    assert(A1in == A2in)
    assert(B1in == B2in)
    assert(A1in != B1in)
    passed()
else:
    print("no OpenCL support, skipped", file=sys.stderr)

**Step 2:** Execute kernel $k$ 4 times: $k(A_{1in}) \rightarrow A_{1out}$, $k(B_{1in})
   \rightarrow B_{1out}$, $k(A_{2in}) \rightarrow A_{2out}$, $k(B_{2in})
   \rightarrow B_{2out}$.

In [ ]:
if has_opencl():
    k = partial(driver, queue)
    A1out = k(A1in)
    B1out = k(B1in)
    A2out = k(A2in)
    B2out = k(B2in)
else:
    print("no OpenCL support, skipped", file=sys.stderr)

**Step 3:**  Assert:

* $A_{1out} \ne A_{1in}$ and $B_{1out} \ne B_{1in}$, else $k$ has no output (for these inputs).

* $A_{1out} \ne B_{1out}$ and $A_{2out} \ne B_{2out}$, else $k$ is input insensitive (for these inputs).

* $A_{1out}=A_{2out}$ and $B_{1out}=B_{2out}$, else $k$ is non-deterministic.

In [ ]:
if has_opencl():
    # outputs must be consistent across runs:
    assert(A1out == A2out)
    assert(B1out == B2out)

    # outputs must depend on inputs:
    if any(not x.is_const for x in driver.prototype.args):
        assert(A1out != B1out)

    # outputs must be different from inputs:
    assert(A1in != A1out)
    assert(B1in != B1out)
else:
    print("no OpenCL support, skipped", file=sys.stderr)

Equality checks for floating point values are performed with an appropriate
epsilon to accommodate rounding errors, and a timeout threshold is also used to
catch kernels which are non-terminating. Our method is based on random
differential testing, though we emphasize that this is not a general purpose
approach and is tailored specifically for our use case. For example, we
anticipate a false positive rate for kernels with subtle sources of
non-determinism which more thorough methods may expose, however we deemed such
methods unnecessary for our purpose of performance modeling.

Examples of programs which are our dynamic checker catches:

In [ ]:
driver = clgen.cldrive.KernelDriver(ctx, """
__kernel void A(__global int* a) {}
""")

try:
    driver.validate(queue, 8192)
except clgen.cldrive.E_NO_OUTPUTS:
    print("caught kernel with no outputs")
except clgen.cldrive.OpenCLNotSupported:
    print("no OpenCL support, skipped", file=sys.stderr)

In [ ]:
driver = clgen.cldrive.KernelDriver(ctx, """
__kernel void A(__global int* a) {
  int b = get_global_id(0);
  a[b] = 0;
}""")

try:
    driver.validate(queue, 8192)
except clgen.cldrive.E_INPUT_INSENSITIVE:
    print("caught input-insensitive kernel")
except clgen.cldrive.OpenCLNotSupported:
    print("no OpenCL support, skipped", file=sys.stderr)

In [ ]:
driver = clgen.cldrive.KernelDriver(ctx, """
__kernel void A(__global float* a) {
  int b = get_global_id(0);

  a[b] = a[b-1] + a[b+1]; /* race condition */
}""")

try:
    driver.validate(queue, 8192)
except clgen.cldrive.E_NONDETERMINISTIC:
    print("caught non-deterministic kernel")
except clgen.cldrive.OpenCLNotSupported:
    print("no OpenCL support, skipped", file=sys.stderr)

# 6. Evaluation of Synthetic Programs

In this section we evaluate the quality of programs synthesized by CLgen
by their likeness to hand-written code, and discuss limitations of the
synthesis and execution pipeline.

## 6.1. Likeness to hand-written code

Judging whether a source code was written by a human is a challenging task for a
machine, so we adopt a methodology from machine learning research based on the
*Turing Test*. We reason that if the output of CLgen is human like code, then a
human judge will be unable to distinguish it from hand-written code.

We devised a double blind test in which 15 volunteer OpenCL developers from
industry and academia were shown 10 OpenCL kernels each. Participants were
tasked with judging whether, for each kernel, they believed it to have been
written by hand or by machine. Kernels were randomly selected for each
participant from two equal sized pools of synthetically generated and hand-
written code from GitHub.

The participants were divided into two groups, with 10 of them receiving code
generated by CLgen, and 5 of them acting as a control group, receiving code
generated by CLSmith, a program generator for differential testing.

In [ ]:
data = pd.read_csv("../data/turing-test.csv")
data["Score (%)"] = (data["True Positive"] + data["True Negative"]) * 10
data

Analysis of participant’s answers:

In [ ]:
clgen_data   = data[data["Synthetic Source"].str.contains("CLgen")]
clsmith_data = data[data["Synthetic Source"].str.contains("CLSmith")]

print("Mean CLgen score:  ", round(  clgen_data["Score (%)"].mean(), 1), "%, stdev:",
      round(  clgen_data["Score (%)"].std(), 1), "%")
print("Mean CLSmith score:", round(clsmith_data["Score (%)"].mean(), 1), "%, stdev:",
      round(clsmith_data["Score (%)"].std(), 1), "%")

This an unsurprising outcome as generated programs for
testing have multiple “tells”, for example, their only input is a single `ulong`
pointer. This demonstrates that CLgen code is indistinguishable from hand-written
programs, with human judges scoring no better than random chance.

An online game of this test is available [here](http://humanorrobot.uk/game/?g=opencl&m=nitt).

## 6.2. Limitations

Our new approach enables the synthesis of more human-like programs than current
state of the art program generators, and without the expert guidance required by
template based generators, but it has limitations. Currently we only run single-
kernel benchmarks, and our method of seeding the language models with the start
of a function means that we cannot support user defined types, or calls to user-
defined functions. The first limitation will be overcome by extending the host
driver to explore multi-kernel schedules and interleaving of kernel executions.
The second limitation can be addressed through recursive program synthesis,
whereby a call to a user-defined function or type will trigger candidate
functions and types to be synthesized.

# 7. Experimental Methodology

## 7.1. Experimental Setup 

#### Predictive Model

We reproduce the predictive model from Grewe et al. (2013). The predictive model
is used to determine the optimal mapping of a given OpenCL kernel to either a
GPU or CPU. It uses supervised learning to construct a decision tree with a
combination of static and dynamic kernel features extracted from source code and
the OpenCL runtime.

In [ ]:
def features_to_dict(path):
    import clgen.features
    buf = StringIO()
    clgen.features.feature_headers(file=buf)  # feature names
    keys = buf.getvalue().rstrip().split(",")
    buf = StringIO()
    clgen.features.features(path, file=buf)  # extract features
    values = buf.getvalue().rstrip().split(",")
    return dict((k, v) for k, v in zip(keys, values))

# kernel to get features from:
kernel = """
__kernel void foobar(__global float *a, __global float *b, const int n) {
    unsigned int i = get_global_id(0);
    if (i < n)
        b[i] += a[i] * 2; 
}
"""
    
with open("../data/kernel.cl", "w") as outfile:
    print(kernel, file=outfile)

DictTable(features_to_dict("../data/kernel.cl"))

#### Benchmarks

As in Grewe et al., we test our model on the NAS Parallel Benchmarks (NPB). We
use the hand-optimized OpenCL implementation of Seo (2011). In Grewe et al. the
authors augment the training set of the predictive model with 47 additional
kernels taken from 4 GPGPU benchmark suites. To more fully sample the program
space, we use a much larger collection of 142 programs. These additional
programs are taken from all 7 of the most frequently used benchmark suites
identified in the motivation.

In [ ]:
benchmarks = sorted(set(amd_benchmarks["benchmark"]))

print(len(benchmarks), "benchmark kernels:")
for benchmark in benchmarks:
    print(" ", benchmark)

We synthesized 1,000 kernels with CLgen to use as additional benchmarks.

In [ ]:
clgen_kernels = fs.ls("../data/clgen-1000/kernels")

print(len(clgen_kernels), "CLgen kernels:")
for kernel in clgen_kernels:
    print(" ", kernel)

#### Platforms

We evaluate our approach on two 64-bit CPU-GPU systems. One system has an AMD
GPU and uses OpenSUSE 12.3; the other is equipped with an NVIDIA GPU and uses
Ubuntu 16.04. Both platforms were unloaded.

In [ ]:
import clgen.clutil
clgen.clutil.platform_info()

#### Datasets

The NPB and Parboil benchmark suites are packaged with multiple datasets. We use
all of the packaged datasets (5 per program in NPB, 1-4 per program in Parboil).
For all other benchmarks, the default datasets are used. We configured the CLgen
host driver to synthesize payloads between 128B-130MB, approximating that of the
dataset sizes found in the benchmark programs.

## 7.2. Methodology

We replicated the methodology of Grewe et al. Each experiment is repeated five
times and the average execution time is recorded. The execution time includes
both device compute time and the data transfer overheads.

We use *leave-one-out cross-validatation* to evaluate predictive models. For
each benchmark, a model is trained on data from all other benchmarks and used to
predict the mapping for each kernel and dataset in the excluded program. We
repeat this process with and without the addition of synthetic benchmarks in the
training data. We do not test model prediction on synthetic benchmarks.

Reproducing experiments on the AMD APP SDK benchmark suites:

In [ ]:
if can_reproduce_experiments():
    !rm -fv ../data/benchmarks/*.csv
    !cd benchmarks && ./mkdata
else:
    print("platform does not have the requirements to run this experiment", file=sys.stderr)

In [ ]:
data = None
if can_reproduce_experiments():
    data = pd.read_csv("../data/benchmarks/training.csv")
    
    ax = sns.barplot(x="benchmark", y="speedup", data=data)
    plt.ylabel("Speedup best over worst mapping")
    plt.xlabel("AMD SDK Benchmark kernels")
    plt.axhline(y=1, color="k", lw=1)  # speedup line
    plt.setp(ax.get_xticklabels(), rotation=90)  # rotate x ticks
    ax.set_xticklabels([shortbenchmark(x.get_text()) for x in ax.get_xticklabels()])
    viz.finalise(figsize=(9,4))
else:
    print("no new data, skipped", file=sys.stderr)
data

Reproducing experiments on a 1% subset of CLgen kernels:

In [ ]:
if can_reproduce_experiments():
    !rm -fv ../data/clgen-10/*.csv
    !cd bin && ./mkdata
else:
    print("platform does not have the requirements to run this experiment", file=sys.stderr)

In [ ]:
data = None
if can_reproduce_experiments():
    data = pd.read_csv("../data/clgen-10/training.csv")
    
    ax = sns.barplot(x="benchmark", y="speedup", ci=95, data=data)
    plt.ylabel("Speedup (95% CI across datasets)")
    plt.xlabel("CLgen kernels")
    plt.axhline(y=1, color="k", lw=1)  # speedup line
    ax.set_xticklabels(range(1, len(data) + 1))
    viz.finalise(figsize=(9,4))
else:
    print("no new data, skipped", file=sys.stderr)
data

# 8. Experimental Results

We evaluate the effectiveness of our approach on two heterogeneous systems. We
first compare the performance of a state of the art predictive model Grewe et
al. with and without the addition of synthetic benchmarks, then show how the
synthetic benchmarks expose weaknesses in the model’s design and how these can
be addressed to develop a better model. Finally we compare the ability of CLgen
to explore the program feature space against a state of the art program
generator.

## 8.1. Performance Evaluation

Performance data from 1,000 CLgen kernels:

In [ ]:
amd_clgen = pd.read_csv("../data/amd-clgen.csv")
amd_clgen

In [ ]:
nvidia_clgen = pd.read_csv("../data/nvidia-clgen.csv")
nvidia_clgen

**Figure 7:** Speedup of programs using *Grewe et al.* predictive model with and without synthetic benchmarks.

In [ ]:
def speedups_with_clgen(platform):
    # datasets: B - benchmarks, S - synthetics, BS - benchmarks + synthetics:
    B = pd.read_csv("../data/{platform}-benchmarks.csv".format(platform=platform))
    B["group"] = ["B"] * len(B)

    S = pd.read_csv("../data/{platform}-clgen.csv".format(platform=platform))
    S["group"] = ["S"] * len(S)

    BS = pd.concat((B, S))

    # find the ZeroR. This is the device which is most frequently optimal
    Bmask = B[B["benchmark"].str.contains("npb-3.3-")]
    zeror = Counter(Bmask["oracle"]).most_common(1)[0][0]
    zeror_runtime = "runtime_" + zeror.lower()

    # get the names of the benchmarks, in the form: $suite-$version-$benchmark
    benchmark_names = sorted(set([
        re.match(r"^([^0-9]+-[0-9\.]+-[^-]+)-", b).group(1)
        for b in B["benchmark"] if b.startswith("npb-")
    ]))

    B_out, BS_out = [], []
    for benchmark in benchmark_names:
        clf = cgo13.model()
        features = get_cgo13_features
        # cross validate on baseline
        B_out += cgo13.leave_one_benchmark_out(clf, features, B, benchmark)
        # reset model
        clf = cgo13.model()
        # repeate cross-validation with synthetic kernels
        BS_out += cgo13.leave_one_benchmark_out(clf, features, BS, benchmark)

    # create results frame
    R_out = []
    for b, bs in zip(B_out, BS_out):
        # get runtimes of device using predicted device
        b_p_runtime = b["runtime_" + b["p"].lower()]
        bs_p_runtime = bs["runtime_" + bs["p"].lower()]

        # speedup is the ratio of runtime using the predicted device
        # over runtime using ZeroR device
        b["p_speedup"] = b_p_runtime / b[zeror_runtime]
        bs["p_speedup"] = bs_p_runtime / bs[zeror_runtime]

        # get the group label, in the form $benchmark.$dataset
        group = re.sub(r"[^-]+-[0-9\.]+-([^-]+)-.+", r"\1",
                       b["benchmark"]) + "." + b["dataset"]
        b["group"] = group
        bs["group"] = group

        # set the training data type
        b["training"] = "Grewe et al."
        bs["training"] = "w. CLgen"

        R_out.append(b)
        R_out.append(bs)

    R = pd.DataFrame(R_out)

    b_mask = R["training"] == "Grewe et al."
    bs_mask = R["training"] == "w. CLgen"

    B_speedup = labmath.mean(R[b_mask].groupby(["group"])["p_speedup"].mean())
    BS_speedup = labmath.mean(R[bs_mask].groupby(["group"])["p_speedup"].mean())

    print("Results on {}:".format(platform.upper()))
    print("  #. benchmarks:                  ",
          len(set(B["benchmark"])), "kernels,", len(B), "observations")
    print("  #. synthetic:                   ",
          len(set(S["benchmark"])), "kernels,", len(S), "observations")
    print()
    print("  ZeroR device:                    {}".format(zeror))
    print()
    print("  Speedup of Grewe et al.:         {:.2f} x".format(B_speedup))
    print("  Speedup w. CLgen:                {:.2f} x".format(BS_speedup))

    R = R.append({  # average bars
        "group": "Average",
        "p_speedup": B_speedup,
        "training": "Grewe et al."
    }, ignore_index=True)
    R = R.append({
        "group": "Average",
        "p_speedup": BS_speedup,
        "training": "w. CLgen"
    }, ignore_index=True)

    R["p_speedup"] -= 1  # negative offset so that bars start at 1

    # colors
    palette = sns.cubehelix_palette(len(set(R["training"])),
                                    rot=-.4, light=.85, dark=.35)

    ax = sns.barplot(
        x="group", y="p_speedup", data=R, ci=None, hue="training",
        palette=palette)
    plt.ylabel("Speedup")
    plt.xlabel("")

    plt.axhline(y=0, color="k", lw=1)  # speedup line
    plt.axvline(x=plt.xlim()[1] - 1, color="k", lw=1, linestyle="--")  # average line

    ax.get_legend().set_title("")  # no legend title
    plt.legend(loc='upper right')
    ax.get_legend().draw_frame(True)

    # plot shape and size
    figsize = (9, 2.2)
    if platform == "nvidia":
        typecast = int; plt.ylim(-1, 16)
    else:
        typecast = float

    # counter negative offset:
    ax.set_yticklabels([typecast(i) + 1 for i in ax.get_yticks()])

    plt.setp(ax.get_xticklabels(), rotation=90)

    viz.finalise(figsize=figsize, tight=True)

speedups_with_clgen("amd")
speedups_with_clgen("nvidia")

Speedups of the *Grewe et al. *predictive model over the NAS Parallel Benchmark
suite with and without the addition of synthesized benchmarks for training.
Speedups are calculated relative to the best single-device mapping for each
experimental platform, which is CPU-only for AMD and GPU-only for NVIDIA. The
fine grained coverage of the feature space which synthetic benchmarks provide
improves performance dramatically for the NAS benchmarks. Across both systems,
we achieve an average speedup of 2.42x with the addition of synthetic
benchmarks, with prediction improvements over the baseline for 62.5% of
benchmarks on AMD and 53.1% on NVIDIA.

The strongest performance improvements are on NVIDIA with the `FT` benchmark, a
benchmark which suffers greatly under a single-device mapping. However, the
performance on AMD for the same benchmark slightly degrades after adding the
synthetic benchmarks, which we address in the next section.

Repeat this experiment on the small subset of data we produced earlier:

In [ ]:
if can_reproduce_experiments():
    # datasets: B - benchmarks, S - synthetics, BS - benchmarks + synthetics:
    B = pd.read_csv("../data/benchmarks/training.csv")
    B["group"] = ["B"] * len(B)

    S = pd.read_csv("../data/clgen-10/training.csv")
    S["group"] = ["S"] * len(S)

    BS = pd.concat((B, S))

    # find the ZeroR. This is the device which is most frequently optimal
    zeror = Counter(B["oracle"]).most_common(1)[0][0]
    zeror_runtime = "runtime_" + zeror.lower()

    # get the names of the benchmarks, in the form: $suite-$version-$benchmark
    benchmark_names = sorted(set([
        re.match(r"^([^0-9]+-[0-9\.]+-[^-]+)-", b).group(1)
        for b in B["benchmark"]
    ]))

    B_out, BS_out = [], []
    for benchmark in benchmark_names:
        clf = cgo13.model()
        features = get_cgo13_features
        # cross validate on baseline
        B_out += cgo13.leave_one_benchmark_out(clf, features, B, benchmark)
        # reset model
        clf = cgo13.model()
        # repeate cross-validation with synthetic kernels
        BS_out += cgo13.leave_one_benchmark_out(clf, features, BS, benchmark)

    # create results frame
    R_out = []
    for b, bs in zip(B_out, BS_out):
        # get runtimes of device using predicted device
        b_p_runtime = b["runtime_" + b["p"].lower()]
        bs_p_runtime = bs["runtime_" + bs["p"].lower()]

        # speedup is the ratio of runtime using the predicted device
        # over runtime using ZeroR device
        b["p_speedup"] = b_p_runtime / b[zeror_runtime]
        bs["p_speedup"] = bs_p_runtime / bs[zeror_runtime]

        # get the group label, in the form $benchmark.$dataset
        group = escape_benchmark_name(b["benchmark"])
        b["group"] = group
        bs["group"] = group

        # set the training data type
        b["training"] = "Grewe et al."
        bs["training"] = "w. CLgen"

        R_out.append(b)
        R_out.append(bs)

    R = pd.DataFrame(R_out)
    b_mask = R["training"] == "Grewe et al."
    bs_mask = R["training"] == "w. CLgen"

    B_speedup = labmath.mean(R[b_mask].groupby(["group"])["p_speedup"].mean())
    BS_speedup = labmath.mean(R[bs_mask].groupby(["group"])["p_speedup"].mean())

    print("Results:")
    print("  #. benchmarks:                  ",
          len(set(B["benchmark"])), "kernels,", len(B), "observations")
    print("  #. synthetic:                   ",
          len(set(S["benchmark"])), "kernels,", len(S), "observations")
    print()
    print("  ZeroR device:                    {}".format(zeror))
    print()
    print("  Speedup of Grewe et al.:         {:.2f} x".format(B_speedup))
    print("  Speedup w. CLgen:                {:.2f} x".format(BS_speedup))

    R = R.append({  # average bars
        "group": "Average",
        "p_speedup": B_speedup,
        "training": "Grewe et al."
    }, ignore_index=True)
    R = R.append({
        "group": "Average",
        "p_speedup": BS_speedup,
        "training": "w. CLgen"
    }, ignore_index=True)

    R["p_speedup"] -= 1  # negative offset so that bars start at 1

    # colors
    palette = sns.cubehelix_palette(len(set(R["training"])),
                                    rot=-.4, light=.85, dark=.35)

    ax = sns.barplot(
        x="group", y="p_speedup", data=R, ci=None, hue="training",
        palette=palette)
    plt.ylabel("Speedup")
    plt.xlabel("")

    plt.axhline(y=0, color="k", lw=1)  # speedup line
    plt.axvline(x=plt.xlim()[1] - 1, color="k", lw=1, linestyle="--")  # average line

    ax.get_legend().set_title("")  # no legend title
    plt.legend(loc='upper right')
    ax.get_legend().draw_frame(True)

    # plot shape and size
    figsize = (7, 3.2)
    typecast = float

    # counter negative offset:
    ax.set_yticklabels([typecast(i) + 1 for i in ax.get_yticks()])

    plt.setp(ax.get_xticklabels(), rotation=90)

    viz.finalise(figsize=figsize, tight=True)
else:
    print("no new data, skipped", file=sys.stderr)

## 8.3. Extending the Predictive Model

Feature designers are bound to select as features only properties which are
significant for the sparse benchmarks they test on, which can limit a model’s
ability to generalize over a wider range of programs. We found this to be the
case with the *Grewe et al.* model. The addition of automatically generated
programs exposed two distinct cases where the model failed to generalize as a
result of overspecializing to the NPB suite.

The first case is that `F3` is sparse on many programs. This a result of the NPB
implementation’s heavy exploitation of local memory buffers and the method by
which they combined features (we speculate this was a necessary dimensionality
reduction in the presence of sparse training programs). To counter this we
extended the model to use the raw features in addition to the combined features.

Distribution of feature values:

In [ ]:
F = [  # features
    amd_clgen["F1:transfer/(comp+mem)"],
    amd_clgen["F2:coalesced/mem"],
    amd_clgen["F3:(localmem/mem)*avgws"],
    amd_clgen["F4:comp/mem"]
]
nres = len(F[0])

plt.subplots(1, 4)
for i in range(len(F)):
    plt.subplot(1, 4, i + 1)
    plt.hist(F[i], bins=20)
    plt.xlim(0,)
    plt.xlabel("feature " + str(i + 1))
    nzeros = sum([1 for x in F[i] if x == 0])
    print("ratio zero values feature {}: {:.3f} %".format(i + 1, (nzeros / nres) * 100))

viz.finalise(figsize=(9,3))

The second case is that some of our generated programs had identical feature
values as in the benchmark set, but had different *behavior* (i.e. optimal
mappings):

In [ ]:
def same_features_different_behaviour(a, b, getfeatures):
    """returns true if features match but oracle device differs"""
    af, bf = getfeatures(a), getfeatures(b)
    ab, bb = a["oracle"], b["oracle"]
    return ab != bb and np.array_equal(af, bf)

print("Number of instances where CLgen kernels have identical features to benchmarks but different behaviour:")
matches, i, imax = defaultdict(int), 0, len(amd_benchmarks)
for a in amd_benchmarks.to_dict('records'):
    i += 1
    print("\r", i, "of", imax, a["benchmark"], end="")
    for b in amd_clgen.to_dict('records'):
        if same_features_different_behaviour(a, b, cgo13.static_features):
            matches[a["benchmark"]] += 1

print("\r" + clgen.format_json(matches))

We found this to be caused by the lack of discriminatory features for branching,
since the NPB programs are implemented in a manner which aggressively minimized
branching. To counter this we extended the predictive model with an additional
feature with a static count of branching operations in a kernel.

Speedups of our extended model across all seven of the benchmark suites used in Section 2:

**Figure 8:** Speedups of predictions using our extended model over *Grewe et al.* on both experimental platforms.

In [ ]:
def compare_clfs(clf1, get_features1, clf2, get_features2, D1, D2, benchmark):
    """cross-validate across all benchmarks using CGO13 model and our own, with
    and without synthetic benchmarks. Report per-platform speedup of our model
    over CGO13"""
    test1_mask = D1["benchmark"].str.contains(r"^" + benchmark)
    test2_mask = D2["benchmark"].str.contains(r"^" + benchmark)
    assert(len(D1[test1_mask]) == len(D2[test2_mask]))

    # create data masks. For training we exclude all results from benchmark
    train1_mask = ~test1_mask
    train2_mask = ~test2_mask

    # create training and testing data
    X1_train = get_features1(D1.loc[train1_mask])
    X2_train = get_features2(D2.loc[train2_mask])
    y1_train = cgo13.getlabels(D1[train1_mask])
    y2_train = cgo13.getlabels(D2[train2_mask])

    D1_test = D1[test1_mask]
    D2_test = D2[test2_mask]
    X1_test = get_features1(D1.loc[test1_mask])
    X2_test = get_features2(D2.loc[test2_mask])
    y1_test = cgo13.getlabels(D1_test)
    y2_test = cgo13.getlabels(D2_test)

    clf1.fit(X1_train, y1_train)  # train classifiers
    clf2.fit(X2_train, y2_train)

    predicted1 = clf1.predict(X1_test)  # make predictions
    predicted2 = clf2.predict(X2_test)

    D_out = []
    for d, y, p1, p2 in zip(D1_test.to_dict('records'), y1_test,
                            predicted1, predicted2):
        d["p1"], d["p2"] = p1, p2
        D_out.append(d)

    return D_out  # return a list of dicts


aB = pd.read_csv("../data/amd-benchmarks.csv")
aB["synthetic"] = np.zeros(len(aB))
bB = pd.read_csv("../data/nvidia-benchmarks.csv")
bB["synthetic"] = np.zeros(len(bB))
B = pd.concat((aB, bB))

aS = pd.read_csv("../data/amd-clgen.csv")
aS["synthetic"] = np.ones(len(aS))
bS = pd.read_csv("../data/nvidia-clgen.csv")
bS["synthetic"] = np.ones(len(bS))
S = pd.concat((aS, bS))

aBS = pd.concat((aB, aS))
bBS = pd.concat((bB, bS))
BS = pd.concat((B, S))

assert(len(B) == len(aB) + len(bB))  # sanity checks
assert(len(S) == len(aS) + len(bS))
assert(len(BS) == len(aBS) + len(bBS))

# get benchmark names: <suite>-<benchmark>
benchmark_names = sorted(set([
    re.match(r"^([^0-9]+-[0-9\.]+-[^-]+)", b).group(1)
    for b in B["benchmark"]
]))

# perform cross-validation
B_out = []
for i, benchmark in enumerate(benchmark_names):
    print("\ranalyzing", i + 1, benchmark, end="")
    cgo13_clf, our_clf = cgo13.model(), get_our_model()
    cgo13_features, our_features = get_cgo13_features, get_our_features

    # cross validate on Grewe et al. and our model
    tmp = compare_clfs(cgo13_clf, cgo13_features, our_clf, our_features,
                       aBS, aBS, benchmark)
    for d in tmp: d["platform"] = "AMD Tahiti 7970"
    B_out += tmp

    # reset models
    cgo13_clf, our_clf = cgo13.model(), get_our_model()

    # same as before, on other platform:
    tmp = compare_clfs(cgo13_clf, cgo13_features, our_clf, our_features,
                       bBS, bBS, benchmark)
    for d in tmp: d["platform"] = "NVIDIA GTX 970"
    B_out += tmp
print()

# create results frame
R_out = []
# get runtimes of device using predicted device
for b in B_out:
    p1_runtime = b["runtime_" + b["p1"].lower()]
    p2_runtime = b["runtime_" + b["p2"].lower()]

    # speedup is the ratio of runtime using our predicted device
    # over runtime using CGO13 predicted device.
    b["p_speedup"] = p2_runtime / p1_runtime

    # get the benchmark name
    b["group"] = escape_benchmark_name(b["benchmark"])

    R_out.append(b)
R = pd.DataFrame(R_out)

improved = R[R["p_speedup"] > 1]

Amask = R["platform"] == "AMD Tahiti 7970"
Bmask = R["platform"] == "NVIDIA GTX 970"
a = R[Amask]
b = R[Bmask]

a_speedups = a.groupby(["group"])["p_speedup"].mean()
b_speedups = b.groupby(["group"])["p_speedup"].mean()

a_speedup = labmath.mean(a_speedups)
b_speedup = labmath.mean(b_speedups)

assert(len(R) == len(a) + len(b))  # sanity-check

print("  #. benchmarks:          ",
      len(set(B["benchmark"])), "kernels,", len(B), "observations")
print("  #. synthetic:           ",
      len(set(S["benchmark"])), "kernels,", len(S), "observations")
print()
print("  Speedup on AMD:          {:.2f} x".format(a_speedup))
print("  Speedup on NVIDIA:       {:.2f} x".format(b_speedup))

palette = sns.cubehelix_palette(
    len(set(R["platform"])), start=4, rot=.8, light=.8, dark=.3)

R = R.append({  # average bars
    "group": "Average",
    "p_speedup": a_speedup,
    "platform": "AMD Tahiti 7970"
}, ignore_index=True)
R = R.append({
    "group": "Average",
    "p_speedup": b_speedup,
    "platform": "NVIDIA GTX 970"
}, ignore_index=True)

R["p_speedup"] -= 1  # negative offset so that bars start at 1

ax = sns.barplot(x="group", y="p_speedup", hue="platform", data=R,
                 palette=palette, ci=None)

plt.ylabel("Speedup over Grewe et al."); plt.xlabel("")

plt.axhline(y=0, color="k", lw=1)
plt.axvline(x=plt.xlim()[1] - 1, color="k", lw=1, linestyle="--")
plt.ylim(-1, 9)
plt.setp(ax.get_xticklabels(), rotation=90)  # rotate x ticks
ax.get_legend().set_title("")  # legend
plt.legend(loc='upper right')

# counter negative offset
ax.set_yticklabels([int(i) + 1 for i in ax.get_yticks()])

ax.get_legend().draw_frame(True)

viz.finalise(figsize=(9, 4), tight=True)

Model performance, even on this tenfold increase of benchmarks, is good. There
are three benchmarks on which the model performs poorly: `gemmv`, `hotspot`, and
`pathfinder`. Each of those programs make heavy use of loops, which we believe
the static code features of the model fail to capture. This could be addressed
by extracting dynamic instruction counts using profiling, but we considered this
beyond the scope of our work. It is not our goal to perfect the predictive
model, but to show the performance improvements associated with training on
synthetic programs. To this extent, we are successful, achieving average
speedups of $3.56\times$ on AMD and $5.04\times$ on NVIDIA across a very large
test set.

Repeat this experiment using the small subset of experimental data we produced earlier:

In [ ]:
def compare_clfs(clf1, get_features1, clf2, get_features2, D1, D2, benchmark):
    """cross-validate across all benchmarks using CGO13 model and our own, with
    and without synthetic benchmarks. Report per-platform speedup of our model
    over CGO13"""
    test1_mask = D1["benchmark"].str.contains(r"^" + benchmark)
    test2_mask = D2["benchmark"].str.contains(r"^" + benchmark)
    assert(len(D1[test1_mask]) == len(D2[test2_mask]))

    # create data masks. For training we exclude all results from benchmark
    train1_mask = ~test1_mask
    train2_mask = ~test2_mask

    # create training and testing data
    X1_train = get_features1(D1.loc[train1_mask])
    X2_train = get_features2(D2.loc[train2_mask])
    y1_train = cgo13.getlabels(D1[train1_mask])
    y2_train = cgo13.getlabels(D2[train2_mask])

    D1_test = D1[test1_mask]
    D2_test = D2[test2_mask]
    X1_test = get_features1(D1.loc[test1_mask])
    X2_test = get_features2(D2.loc[test2_mask])
    y1_test = cgo13.getlabels(D1_test)
    y2_test = cgo13.getlabels(D2_test)

    clf1.fit(X1_train, y1_train)  # train classifiers
    clf2.fit(X2_train, y2_train)

    predicted1 = clf1.predict(X1_test)  # make predictions
    predicted2 = clf2.predict(X2_test)

    D_out = []
    for d, y, p1, p2 in zip(D1_test.to_dict('records'), y1_test,
                            predicted1, predicted2):
        d["p1"], d["p2"] = p1, p2
        D_out.append(d)

    return D_out  # return a list of dicts

if can_reproduce_experiments():
    B = pd.read_csv("../data/benchmarks/training.csv")
    B["synthetic"] = np.zeros(len(B))

    S = pd.read_csv("../data/clgen-10/training.csv")
    S["synthetic"] = np.ones(len(S))

    BS = pd.concat((B, S))

    assert(len(BS) == len(B) + len(S))

    # get benchmark names: <suite>-<benchmark>
    benchmark_names = sorted(set([
        re.match(r"^([^0-9]+-[0-9\.]+-[^-]+)", b).group(1)
        for b in B["benchmark"]
    ]))

    # perform cross-validation
    B_out = []
    for i, benchmark in enumerate(benchmark_names):
        print("\ranalyzing", i + 1, benchmark, end="")
        cgo13_clf, our_clf = cgo13.model(), get_our_model()
        cgo13_features, our_features = get_cgo13_features, get_our_features

        # cross validate on Grewe et al. and our model
        tmp = compare_clfs(cgo13_clf, cgo13_features, our_clf, our_features,
                           BS, BS, benchmark)
        B_out += tmp
    print()

    # create results frame
    R_out = []
    # get runtimes of device using predicted device
    for b in B_out:
        p1_runtime = b["runtime_" + b["p1"].lower()]
        p2_runtime = b["runtime_" + b["p2"].lower()]

        # speedup is the ratio of runtime using our predicted device
        # over runtime using CGO13 predicted device.
        b["p_speedup"] = p2_runtime / p1_runtime

        # get the benchmark name
        b["group"] = escape_benchmark_name(b["benchmark"])

        R_out.append(b)
    R = pd.DataFrame(R_out)

    improved = R[R["p_speedup"] > 1]

    speedups = R.groupby(["group"])["p_speedup"].mean()
    speedup = labmath.mean(speedups)

    print("  #. benchmarks:          ",
          len(set(B["benchmark"])), "kernels,", len(B), "observations")
    print("  #. synthetic:           ",
          len(set(S["benchmark"])), "kernels,", len(S), "observations")
    print()
    print("  Speedup:                 {:.2f} x".format(speedup))

    palette = sns.cubehelix_palette(1, start=4, rot=.8, light=.8, dark=.3)

    R = R.append({  # average bar
        "group": "Average",
        "p_speedup": speedup
    }, ignore_index=True)

    R["p_speedup"] -= 1  # negative offset so that bars start at 1

    ax = sns.barplot(x="group", y="p_speedup", data=R,
                     palette=palette, ci=None)

    plt.ylabel("Speedup over Grewe et al."); plt.xlabel("")

    plt.axhline(y=0, color="k", lw=1)
    plt.axvline(x=plt.xlim()[1] - 1, color="k", lw=1, linestyle="--")
    plt.ylim(-1, 9)
    plt.setp(ax.get_xticklabels(), rotation=90)  # rotate x ticks

    # counter negative offset
    ax.set_yticklabels([int(i) + 1 for i in ax.get_yticks()])

    viz.finalise(figsize=(7, 3.7), tight=True)
else:
    print("no new data, skipped", file=sys.stderr)

## 8.3. Comparison of Source Features

As demonstrated in Section 2, the predictive quality of a model for a given
point in the feature space is improved with the addition of observations from
neighboring points. By producing thousands of artificial programs modeled on the
structure real OpenCL programs, CLgen is able to consistently and automatically
generate programs which are close in the feature space to the benchmarks which
we are testing on.

To quantify this effect we use the static code features, plus the branching
feature discussed in the previous subsection, to measure the number of CLgen
kernels generated with the same feature values as those of the benchmarks we
examined in the previous subsections. We examine only static code features to
allow comparison with the GitHub kernels for which we have no automated method
to execute them and extract runtime features, and CLSmith generated programs.

Plotting the number of matches as a function of the number of kernels:

**Figure 9:** The number of kernels from GitHub, CLSmith, and CLgen with static code features matching the benchmarks. CLgen generates kernels that are closer in the feature space than CLSmith, and can continue to do so long after we have exhausted the extent of the GitHub dataset.

In [ ]:
def uniq_benchmarks(g):
    """return set of unique benchmarks, removes duplicates"""
    uniq, out = set(), []
    for row in g.to_dict('records'):
        key = (row['benchmark'])
        if key not in uniq:
            out.append(row)
            uniq.add(key)
    return pd.DataFrame(out)

np.random.seed(204)

# load datasets
CLgen = pd.read_csv("../data/clgen-features.csv")
GitHub = pd.read_csv("../data/github-features.csv")
CLSmith = pd.read_csv("../data/clsmith-features.csv")

# extract features
Benchmarks_features = cgo13.get_static_features(uniq_benchmarks(amd_benchmarks))
CLgen_features = cgo13.get_static_features(CLgen)
GitHub_features = cgo13.get_static_features(GitHub)
CLSmith_features = cgo13.get_static_features(CLSmith)

stepMin, stepMax = 0, 10000
linStepSize = int((stepMax - stepMin) / 20)
closeness_threshold = 0

steps = np.array(list(range(stepMin, stepMax + linStepSize, linStepSize)),
                 dtype=np.int32)
steps[0] = max(steps[0], 1)

GitHubSteps = [n for n in steps if n <= len(GitHub_features)]
CLSmithSteps = [n for n in steps if n <= len(CLSmith_features)]
CLgenSteps = [n for n in steps if n <= len(CLgen_features)]

numGitHubSteps = len(GitHubSteps)
numCLSmithSteps = len(CLSmithSteps)
numCLgenSteps = len(CLgenSteps)

print("#. GitHub kernels ", GitHubSteps[-1])
print("#. CLSmith kernels", CLSmithSteps[-1])
print("#. CLgen kernels  ", CLgenSteps[-1])

datasets = [
    {
        "name": "GitHub",
        "color": "b",
        "linestyle": ":",
        "steps": GitHubSteps,
        "static_features": GitHub_features,
        "distances": np.zeros(numGitHubSteps, dtype=float),
        "errors": np.zeros(numGitHubSteps, dtype=float)
    },
    {
        "name": "CLSmith",
        "color": "r",
        "linestyle": "--",
        "steps": CLSmithSteps,
        "static_features": CLSmith_features,
        "distances": np.zeros(numCLSmithSteps, dtype=float),
        "errors": np.zeros(numCLSmithSteps, dtype=float)
    },
    {
        "name": "CLgen",
        "color": "g",
        "linestyle": "-",
        "steps": CLgenSteps,
        "static_features": CLgen_features,
        "distances": np.zeros(numCLgenSteps, dtype=float),
        "errors": np.zeros(numCLgenSteps, dtype=float)
    },
]

numIterations = 10  # repeat experiment N times
for dataset in datasets:
    for i, n in enumerate(dataset["steps"]):
        numSteps = len(dataset["steps"])
        print("\r", dataset["name"], i + 1, "of", numSteps, end="")
        r = np.zeros(numIterations, dtype=float)
        for j in range(numIterations):
            np.random.shuffle(dataset["static_features"])
            distances = get_nearest_neighbour_distance(
                dataset["static_features"][:n], Benchmarks_features)
            r[j] = sum([1 if d <= closeness_threshold else 0 for d in distances])
        dataset["distances"][i] = np.mean(r)
        dataset["errors"][i] = np.std(r)
    plt.errorbar(dataset["steps"], dataset["distances"],
                 yerr=dataset["errors"], c=dataset["color"],
                 linestyle=dataset["linestyle"],
                 label=dataset["name"], ecolor="k", capthick=1)
ax = plt.gca()

plt.legend(loc='upper left')
ax.get_legend().set_title("")
ax.get_legend().draw_frame(True)

plt.xlim(stepMin, stepMax)
plt.ylim(0,)
plt.xlabel("#. kernels"); plt.ylabel("#. matches")

viz.finalise(figsize=(4.5, 2))

Out of 10,000 unique CLgen kernels, more than a third have static feature values
matching those of the benchmarks, providing on average 14 CLgen kernels for each
benchmark. This confirms our original intuition: CLgen kernels, by emulating the
way real humans write OpenCL programs, are concentrated in the same area of the
feature space as real programs. Moreover, the number of CLgen kernels we
generate is unbounded, allowing us to continually refine the exploration of the
feature space, while the number of kernels available on GitHub is finite.
CLSmith rarely produces code similar to real-world OpenCL programs, with only
0.53% of the generated kernels have matching feature values with benchmark
kernels. We conclude that the unique contribution of CLgen is its ability to
generate many thousands of programs *that are appropriate for predictive
modeling*.

# 9. Related Work

See the PDF version of this paper.

# 10. Conclusion

The quality of predictive models is bound by the quantity and quality of
programs used for training, yet there is typically only a few dozen common
benchmarks available for experiments. We present a novel tool which is the first
of it’s kind — an entirely probabilistic program generator capable of generating
an unbounded number of human like programs. Our approach applies deep learning
over a huge corpus of publicly available code from GitHub to automatically infer
the semantics and practical usage of a programming language. Our tool generates
programs which to trained eyes are indistinguishable from hand-written code. We
tested our approach using a state of the art predictive model, improving its
performance by a factor of 1.27x. We found that synthetic benchmarks exposed
weaknesses in the feature set which, when corrected, further improved the
performance by 2.66x. In future work we will extend CLgen to synthesize
benchmarks in multiple programming languages, and investigate methods for
performing an automatic directed search of the program space.

In [ ]:
print("Evaluation complete.")